<a href="https://colab.research.google.com/github/Khalil1232/Machine-Learning-ULaval/blob/main/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Devoir 4, Question 2 : Transfert de représentation

# Homework 4, Question 2: Transfer learning

## Code préambule

## Preamble code

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import gzip
import time
import numpy

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
import torchvision.transforms as T
from tqdm import tqdm

import matplotlib
matplotlib.rcParams['figure.figsize'] = (9.0, 7.0)
from matplotlib import pyplot

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def compute_accuracy(model, dataloader, device='cpu'):
    training_before = model.training
    model.eval()
    all_predictions = []
    all_targets = []

    for i_batch, batch in enumerate(dataloader):
        images, targets = batch
        images = images.to(device)
        targets = targets.to(device)
        with torch.no_grad():
            predictions = model(images)
        all_predictions.append(predictions.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

    if all_predictions[0].shape[-1] > 1:
        predictions_numpy = numpy.concatenate(all_predictions, axis=0)
        predictions_numpy = predictions_numpy.argmax(axis=1)
        targets_numpy = numpy.concatenate(all_targets, axis=0)
    else:
        predictions_numpy = numpy.concatenate(all_predictions).squeeze(-1)
        targets_numpy = numpy.concatenate(all_targets)
        predictions_numpy[predictions_numpy >= 0.5] = 1.0
        predictions_numpy[predictions_numpy < 0.5] = 0.0

    if training_before:
        model.train()

    return (predictions_numpy == targets_numpy).mean()

Pour cette question, vous devez programmer un cas de transfert de représentation permettant de réutiliser un réseau existant. Un réseau *ResNet-18* préalablement entraîné sur le jeu d'images naturelles *ImageNet* est utilisé comme modèle source. Ce réseau a été préentraîné sur un jeu de données différent, mais de même nature, soit pour de la reconnaissance d'objets. L'adaptation du réseau original pour la nouvelle tâche s'effectue en remplaçant la tête du réseau (couche de sortie) pour que le réseau puisse fonctionner sur le jeu de données [*Lego Brick*](https://www.kaggle.com/joosthazelzet/lego-brick-images), séparé en un ensemble d'[entraînement](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/lego-train.zip) et de [test](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/lego-test.zip) (attention: 190 Mo au total, les fichiers sont directement disponibles sur PAX). Vous devriez être en mesure d'atteindre de très bonnes performances sur ce jeu en seulement une époque d'entraînement.

En bref, vous devez modifier la dernière couche pleinement connectée du réseau de neurones (couche `fc`) afin de l'adapter au nombre de classes du jeu de données (16 classes ici). De plus, vous devez geler les autres couches du réseau *ResNet-18* se trouvant avant la nouvelle couche pleinement connectée de sortie. Écrivez également la ligne de code nécessaire à l'inférence du réseau dans la méthode `forward`.

For this question, you need to program a representation transfer case to reuse an existing network. A network *ResNet-18* previously trained on the natural image set *ImageNet* is used as the source model. This network has been pre-trained on a different dataset, but of the same nature, i.e. for object recognition. The adaptation of the original network for the new task is done by replacing the head of the network (output layer) so that the network can run on the [*Lego Brick*](https://www.kaggle.com/joosthazelzet/lego-brick-images) dataset, separated into a set of [training](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/lego-train.zip) and [test](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/lego-test.zip) (warning: 190 MB overall, the files are directly available on PAX). You should be able to achieve very good performance on this game in just one training epoch.

In short, you need to modify the last fully connected layer of the neural network (the `fc` layer) to fit the number of classes in the dataset (16 classes here). In addition, you need to freeze the other layers of the *ResNet-18* network before the new fully connected output layer. Also write the line of code needed to infer the network in the `forward` method.

## Q2A
Changez la dernière couche pleinement connectée du réseau de neurones (couche `fc`) afin de l'adapter au nombre de classes du jeu de données (16 classes ici). De plus, gelez les autres couches du réseau *ResNet-18* se trouvant avant la nouvelle couche pleinement connectée de sortie. Écrivez également la ligne de code nécessaire à l'inférence du réseau dans la méthode `forward`.

## Q2A
Change the last fully connected layer of the neural network (the `fc` layer) to fit the number of classes in the dataset (16 classes here). In addition, freeze the other layers of the *ResNet-18* network before the new fully connected output layer. Also write the line of code needed to infer the network in the `forward` method.

### Entrez votre solution à Q2A dans la cellule ci-dessous

### Enter your answer to Q2A in the cell below.

<div class="feedback-cell" style="background: rgba(100 , 100 , 100 , 0.4)">
                <h3>Votre soumission a été enregistrée!</h3>
                <ul>
                    <li>notez qu'il n'y a <strong>pas</strong> de correction automatique pour cet exercice&puncsp;;</li>
                    <li>par conséquent, votre note est <strong>actuellement</strong> zéro&puncsp;;</li>
                    <li>elle sera cependant ajustée par le professeur dès que la correction manuelle sera complétée&puncsp;;</li>
                    <li>vous pouvez soumettre autant de fois que nécessaire jusqu'à la date d'échéance&puncsp;;</li>
                    <li>mais évitez de soumettre inutilement.</li>
                </ul>
            </div>

In [ ]:
class LegoNet(nn.Module):

    def __init__(self, pretrained=False):
        super().__init__()

        # Crée le réseau de neurone pré-entraîné
        # Create the pretrained neural network
        self.model = resnet18(pretrained=pretrained, progress=False)

        # Récupère le nombre de neurones avant la couche de classement
        # Get the number of features before the classification layer
        dim_before_fc = self.model.fc.in_features


        # *** TODO ***
        # Changer la dernière couche pleinement connecté pour avoir le bon
        # nombre de neurones de sortie
        # Change the last fully connected layer to have the right
        # number of output neurons
        # ******
        self.model.fc = nn.Linear(dim_before_fc, 16)

        if pretrained:
            # *** TODO ***
            # Geler les paramètres qui ne font pas partie de la dernière couche fc
            # Conseil: utiliser l'itérateur named_parameters() et la variable requires_grad
            # Freeze parameters that are not part of the last fc layer
            # Tip: use named_parameters() iterator and requires_grad variable
            for name, param in self.model.named_parameters():
                if name != 'fc.weight' and name != 'fc.bias':
                    param.requires_grad = False
            # ******


    def forward(self, x):
        # *** TODO ***
        # Appeler la fonction forward du réseau préentraîné (resnet18) de LegoNet
        # Call the forward function of the pre-trained network (resnet18) of LegoNet
        return self.model(x)
        # ******

### Patron de code réponse à l'exercice Q2A

### Q2A answer code template

In [ ]:
class LegoNet(nn.Module):

    def __init__(self, pretrained=False):
        super().__init__()

        # Crée le réseau de neurone pré-entraîné
        # Create the pretrained neural network
        self.model = resnet18(pretrained=pretrained, progress=False)

        # Récupère le nombre de neurones avant la couche de classement
        # Get the number of features before the classification layer
        dim_before_fc = self.model.fc.in_features


        # *** TODO ***
        # Changer la dernière couche pleinement connecté pour avoir le bon
        # nombre de neurones de sortie
        # Change the last fully connected layer to have the right
        # number of output neurons
        # ******

        if pretrained:
            # *** TODO ***
            # Geler les paramètres qui ne font pas partie de la dernière couche fc
            # Conseil: utiliser l'itérateur named_parameters() et la variable requires_grad
            # Freeze parameters that are not part of the last fc layer
            # Tip: use named_parameters() iterator and requires_grad variable
            pass # Retirer le pass / remove the pass
            # ******


    def forward(self, x):
        # *** TODO ***
        # Appeler la fonction forward du réseau préentraîné (resnet18) de LegoNet
        # Call the forward function of the pre-trained network (resnet18) of LegoNet
        return false
        # ******

## Q2B
Écrivez les lignes de code manquantes pour la préparation de l'entraînement et celles à l'intérieur de la boucle d'entraînement selon deux modes:
1. Entraîner le réseau en exécutant le code **sans** préentraînement, le réseau devrait être entraîné en moins de 30 minutes sur CPU (et quelques minutes sur GPU).
2. Entraîner le réseau en exécutant le code **avec** préentraînement (*fine tuning*).

## Q2B
Write the missing lines of code for the training preparation and those inside the training loop in two modes:
1. Train the network by running the code **without** pre-training, the network should be trained in less than 30 minutes on CPU (and a few minutes on GPU).
2. Train the network by running the code **with** pre-training (*fine tuning*).

### Entrez votre solution à Q2B dans la cellule ci-dessous

### Enter your answer to Q2B in the cell below.

In [ ]:
def train(pretrained):

    # Définit les paramètres d'entraînement
    # Nous vous conseillons ces paramètres, mais vous pouvez les changer
    # Defines the training parameters
    # We recommend these settings, but you can change them
    nb_epoch = 1
    learning_rate = 0.01
    momentum = 0.9
    batch_size = 64

    # Définit les transformations nécessaires pour le chargement du jeu de données
    # Defines the transformations needed to load the dataset
    totensor = T.ToTensor()
    normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    composition = T.Compose([totensor, normalize])

    # Charge le dataset d'entraînement
    # Load the training dataset
    train_set = ImageFolder('/pax/shared/GIF-4101-7005/lego-train', transform=composition)

    # Selectionne 10% du jeu de test aléatoirement pour alléger le calcul
    # Select 10% of the test set randomly to simplify the calculation
    test_set = ImageFolder('/pax/shared/GIF-4101-7005/lego-test', transform=composition)
    idx = numpy.random.randint(0, len(test_set), int(0.1 * len(test_set)))
    test_set.samples = [test_set.samples[i] for i in idx]

    # *** TODO ***
    # Créer les dataloader PyTorch avec la classe DataLoader
    # Create PyTorch dataloaders with the DataLoader class
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

    # Instancier un réseau LegoNet dans une variable nommée "model"
    # Instantiate a LegoNet network in a variable named "model
    # ******
    model = LegoNet(pretrained)

    # Place le réseau au bon endroit, variable DEVICE définit si cuda est utilisé ou non
    # Places the network in the right place, variable DEVICE defines if cuda is used or not
    model.to(DEVICE)

    # *** TODO ***
    # Instancier une fonction d'erreur CrossEntropyLoss et
    # la mettre dans une variable nommée criterion
    # Instantiate a CrossEntropyLoss error function and
    # put it in a variable named criterion
    criterion = nn.CrossEntropyLoss()

    # Instancier l'algorithme d'optimisation SGD
    # Conseil: Filtrez les paramètres non-gelés!
    # Ne pas oublier de lui donner les hyperparamètres d'entraînement :
    # learning rate et momentum!
    # Instantiate the SGD optimization algorithm
    # Tip: Filter out unfrozen parameters!
    # Don't forget to give it the training hyperparameters :
    # learning rate and momentum!
    params = list(filter(lambda x: x.requires_grad, model.parameters()))
    optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=momentum)

    # Mettre le réseau en mode entraînement
    # Set the network in training mode
    # ******
    model.train()

    # Récupère le nombre total de batch pour une époque
    # Retrieves the total number of batches for an epoch.
    total_batch = len(train_loader)

    for i_epoch in tqdm(range(nb_epoch)):
        progress_dataloader = tqdm(train_loader, desc="Epoch {}/{}".format(i_epoch+1, nb_epoch))
        progress_dataloader.set_description("Epoch {}/{}".format(i_epoch+1, nb_epoch))

        train_losses = []
        for batch in progress_dataloader:
            images, targets = batch

            images = images.to(DEVICE)
            targets = targets.to(DEVICE)

            # *** TODO ***
            # Mettre les gradients à zéro
            # Set gradients to zero
            optimizer.zero_grad()

            # Calculer:
            # 1. l'inférence dans une variable "predictions"
            # 2. l'erreur dans une variable "loss"
            # Compute:
            # 1. the inference in a "predictions" variable
            # 2. the error in a "loss" variable
            predictions = model.forward(images)
            loss = criterion(predictions, targets)

            # Rétropropager l'erreur et effectuer une étape d'optimisation
            # Backpropagate the error and perform an optimization step
            # ******
            loss.backward()
            optimizer.step()

            # Ajoute le loss de la batch
            # Adds the batch loss
            train_losses.append(loss.item())

    # Affiche le score à l'écran
    # Display score
    test_acc = compute_accuracy(model, test_loader, DEVICE)
    if pretrained:
        print(' [-] pretrained test acc. {:.6f}%'.format(test_acc * 100))
    else:
        print(' [-] not pretrained test acc. {:.6f}%'.format(test_acc * 100))

    # Libère la cache sur le GPU : *Important sur un cluster de GPU*
    # Free the cache on the GPU: *Important on a GPU cluster*
    torch.cuda.empty_cache()
train(True)
train(False)

100%|██████████| 1/1 [00:14<00:00, 14.32s/it]


 [-] pretrained test acc. 82.103611%


100%|██████████| 1/1 [00:23<00:00, 23.86s/it]


 [-] not pretrained test acc. 45.054945%


### Patron de code réponse à l'exercice Q2B

### Q2B answer code template

In [ ]:
def train(pretrained):

    # Définit les paramètres d'entraînement
    # Nous vous conseillons ces paramètres, mais vous pouvez les changer
    # Defines the training parameters
    # We recommend these settings, but you can change them
    nb_epoch = 1
    learning_rate = 0.01
    momentum = 0.9
    batch_size = 64

    # Définit les transformations nécessaires pour le chargement du jeu de données
    # Defines the transformations needed to load the dataset
    totensor = T.ToTensor()
    normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    composition = T.Compose([totensor, normalize])

    # Charge le dataset d'entraînement
    # Load the training dataset
    train_set = ImageFolder('/pax/shared/GIF-4101-7005/lego-train', transform=composition)

    # Selectionne 10% du jeu de test aléatoirement pour alléger le calcul
    # Select 10% of the test set randomly to simplify the calculation
    test_set = ImageFolder('/pax/shared/GIF-4101-7005/lego-test', transform=composition)
    idx = numpy.random.randint(0, len(test_set), int(0.1 * len(test_set)))
    test_set.samples = [test_set.samples[i] for i in idx]

    # *** TODO ***
    # Créer les dataloader PyTorch avec la classe DataLoader
    # Create PyTorch dataloaders with the DataLoader class

    # Instancier un réseau LegoNet dans une variable nommée "model"
    # Instantiate a LegoNet network in a variable named "model
    # ******

    # Place le réseau au bon endroit, variable DEVICE définit si cuda est utilisé ou non
    # Places the network in the right place, variable DEVICE defines if cuda is used or not
    model.to(DEVICE)

    # *** TODO ***
    # Instancier une fonction d'erreur CrossEntropyLoss et
    # la mettre dans une variable nommée criterion
    # Instantiate a CrossEntropyLoss error function and
    # put it in a variable named criterion

    # Instancier l'algorithme d'optimisation SGD
    # Conseil: Filtrez les paramètres non-gelés!
    # Ne pas oublier de lui donner les hyperparamètres d'entraînement :
    # learning rate et momentum!
    # Instantiate the SGD optimization algorithm
    # Tip: Filter out unfrozen parameters!
    # Don't forget to give it the training hyperparameters :
    # learning rate and momentum!

    # Mettre le réseau en mode entraînement
    # Set the network in training mode
    # ******

    # Récupère le nombre total de batch pour une époque
    # Retrieves the total number of batches for an epoch.
    total_batch = len(train_loader)

    for i_epoch in tqdm(range(nb_epoch)):
        progress_dataloader = tqdm(train_loader, desc="Epoch {}/{}".format(i_epoch+1, nb_epoch))
        progress_dataloader.set_description("Epoch {}/{}".format(i_epoch+1, nb_epoch))

        train_losses = []
        for batch in progress_dataloader:
            images, targets = batch

            images = images.to(DEVICE)
            targets = targets.to(DEVICE)

            # *** TODO ***
            # Mettre les gradients à zéro
            # Set gradients to zero

            # Calculer:
            # 1. l'inférence dans une variable "predictions"
            # 2. l'erreur dans une variable "loss"
            # Compute:
            # 1. the inference in a "predictions" variable
            # 2. the error in a "loss" variable

            # Rétropropager l'erreur et effectuer une étape d'optimisation
            # Backpropagate the error and perform an optimization step
            # ******

            # Ajoute le loss de la batch
            # Adds the batch loss
            train_losses.append(loss.item())

    # Affiche le score à l'écran
    # Display score
    test_acc = compute_accuracy(model, test_loader, DEVICE)
    if pretrained:
        print(' [-] pretrained test acc. {:.6f}%'.format(test_acc * 100))
    else:
        print(' [-] not pretrained test acc. {:.6f}%'.format(test_acc * 100))

   # Libère la cache sur le GPU : *Important sur un cluster de GPU*
   # Free the cache on the GPU: *Important on a GPU cluster*
   torch.cuda.empty_cache()